## Apply all Cell-Health Models to Training and Testing Sets

**Gregory Way, 2019**

In [1]:
import os
import pandas as pd
from joblib import load

from scripts.ml_utils import load_train_test, load_models

In [2]:
%matplotlib inline

In [3]:
def apply_model(model, feature, train_x, test_x):
    """
    Apply model to training and testing matrix
    """
    pred_train_df = (
        pd.DataFrame(model.predict(train_x), columns=["score"])
        .assign(profiles=train_x.index,
                Metadata_data_type="train",
                model=feature)
    )
    pred_test_df = (
        pd.DataFrame(model.predict(test_x), columns=["score"])
        .assign(profiles=test_x.index,
                Metadata_data_type="test",
                model=feature)
    )

    pred_df = pd.concat([pred_train_df, pred_test_df]).reset_index(drop=True)
    return pred_df

def sample_squared_error(scores, y):
    """
    Calculate the squared error per sample depending on model scores
    """
    metadata_cols = [x for x in scores.columns if x.startswith("Metadata_")]
    scores_values = scores.drop(metadata_cols, axis="columns")
    
    all_squared_error = {}
    for cell_health_feature in scores_values.columns:
        y_subset_df = y.loc[:, cell_health_feature].dropna().T
        scores_subset = scores_values.loc[:, cell_health_feature].reindex(y_subset_df.index).T

        squared_error = (y_subset_df - scores_subset) ** 2
        all_squared_error[cell_health_feature] = squared_error
    
    return pd.DataFrame(all_squared_error).reindex(scores.index)

## 1) Load Models and Model Coefficients

For real data and shuffled model data.

In [4]:
model_dict, model_coef = load_models()
shuffle_model_dict, shuffle_model_coef = load_models(shuffle=True)

In [5]:
# Load Metadata Mapping File
data_dir = os.path.join("..", "1.generate-profiles", "data")
file = os.path.join(data_dir, "profile_id_metadata_mapping.tsv")
metadata_df = pd.read_csv(file, sep='\t')

metadata_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


## 2) Load Training and Testing Data

In [6]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(drop_metadata=True)

## 3) Output Model Coefficients

In [7]:
# Extract all model coefficients and output to file
coef_df = pd.DataFrame(model_coef)
coef_df.index = x_test_df.columns
coef_df.index.name = "features"

file = os.path.join("results", "all_model_coefficients.tsv")
coef_df.to_csv(file, sep='\t', index=True)

print(coef_df.shape)
coef_df.head(2)

(1281, 70)


,cc_edu_pos_alexa647_intensity_nucleus_area_mean,cc_cc_n_objects,cc_g2_ph3_pos_n_spots_per_nucleus_area_mean,cc_edu_pos_n_spots_per_nucleus_area_mean,vb_percent_dead_only,vb_percent_dead,cc_polyploid_n_objects,cc_polyploid_high_n_spots_h2ax_mean,cc_edu_pos_high_n_spots_h2ax_mean,cc_cc_g2_ph3_neg_mean,...,cc_all_n_spots_mean,cc_polyploid_n_spots_mean,cc_mitosis_ph3_pos_high_n_spots_h2ax_mean,cc_g2_ph3_pos_n_spots_mean,vb_percent_caspase_dead_only,cc_edu_pos_n_objects,cc_polyploid_n_spots_per_nucleus_area_mean,cc_all_large_notround_polynuclear_mean,cc_g2_ph3_pos_high_n_spots_h2ax_mean,cc_mitosis_ph3_neg_n_spots_mean
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,0.0,0.000000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,...,0.0,-0.0,-0.0,0.0,-0.0,0.000000,-0.0,-0.0,0.0,-0.0
Cells_AreaShape_Center_X,0.0,0.004141,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,...,-0.0,0.0,0.0,-0.0,-0.0,0.006539,0.0,0.0,-0.0,0.0


In [8]:
# Extract all model coefficients and output to file
shuffle_coef_df = pd.DataFrame(shuffle_model_coef)
shuffle_coef_df.index = x_test_df.columns
shuffle_coef_df.index.name = "features"

file = os.path.join("results", "all_model_coefficients_shuffled.tsv")
shuffle_coef_df.to_csv(file, sep='\t', index=True)

print(shuffle_coef_df.shape)
shuffle_coef_df.head(2)

(1281, 70)


,cc_all_large_notround_polynuclear_mean,cc_cc_g1_mean,cc_cc_n_objects,vb_infection_percentage,cc_g2_ph3_pos_n_spots_mean,cc_mitosis_ph3_pos_n_spots_mean,cc_g2_ph3_pos_n_objects,cc_polyploid_n_objects,cc_polynuclear_n_spots_mean,cc_infection_percentage,...,cc_polynuclear_n_spots_per_nucleus_area_mean,cc_polyploid_n_spots_per_nucleus_area_mean,cc_all_high_n_spots_h2ax_mean,cc_edu_pos_alexa647_intensity_nucleus_area_mean,cc_mitosis_ph3_neg_n_spots_mean,cc_g2_ph3_neg_n_spots_mean,cc_g1_high_n_spots_h2ax_mean,cc_all_n_spots_per_nucleus_area_mean,cc_mitosis_ph3_pos_high_n_spots_h2ax_mean,cc_edu_pos_alexa647_intensity_nucleus_area_sum
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.021364,0.0,-0.0
Cells_AreaShape_Center_X,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.0


## 4) Apply all models

For real and shuffled data.

In [9]:
all_scores = []
all_shuffle_scores = []
for cell_health_feature in model_dict.keys():
    # Apply Real Model Classifiers
    model_clf = model_dict[cell_health_feature]
    pred_df = apply_model(model=model_clf,
                          feature=cell_health_feature,
                          train_x=x_train_df,
                          test_x=x_test_df)
    all_scores.append(pred_df)
    
    # Apply Shuffled Model Classifiers
    shuffle_model_clf = shuffle_model_dict[cell_health_feature]
    shuffle_pred_df = apply_model(model=shuffle_model_clf,
                                  feature=cell_health_feature,
                                  train_x=x_train_df,
                                  test_x=x_test_df)
    all_shuffle_scores.append(shuffle_pred_df)

## 5) Concatenate scores with Metadata

In [10]:
# Concatenate real data scores
all_scores = (
    pd.concat(all_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_scores = (
    metadata_df.merge(all_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_scores.index = all_scores.Metadata_profile_id
all_scores = all_scores.drop("Metadata_profile_id", axis="columns")

# Output file
file = os.path.join("results", "all_model_predictions.tsv")
all_scores.to_csv(file, sep='\t', index=True)

print(all_scores.shape)
all_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.204034,0.448810,0.073226,0.204900,0.299412,0.235719,-0.061997,...,0.058389,0.146370,0.035880,0.111507,0.258033,0.098537,0.159442,-0.099482,0.212296,0.411890
profile_1,A549,AKT1-2,train,0.044071,0.454398,0.126608,0.320856,0.027809,0.030007,-0.221132,...,0.096866,0.152027,-0.004406,0.241703,0.243551,0.276340,0.237697,-0.277111,0.247296,0.535884


In [11]:
# Concatenate shuffled data scores
all_shuffle_scores = (
    pd.concat(all_shuffle_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_shuffle_scores = (
    metadata_df.merge(all_shuffle_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_shuffle_scores.index = all_shuffle_scores.Metadata_profile_id
all_shuffle_scores = all_shuffle_scores.drop("Metadata_profile_id", axis="columns")

# Output file
file = os.path.join("results", "all_model_predictions_shuffled.tsv")
all_shuffle_scores.to_csv(file, sep='\t', index=True)

print(all_shuffle_scores.shape)
all_shuffle_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.289201,0.182882,0.261533,-0.160217,0.286642,0.266674,0.145308,...,-0.263365,0.223852,0.150822,0.238369,0.214405,0.245927,0.241706,-0.246256,-0.017288,0.027778
profile_1,A549,AKT1-2,train,0.218454,0.175670,0.261533,-0.160217,0.260630,0.261169,0.150902,...,-0.263365,0.223575,0.160175,0.236691,0.214405,0.245927,0.251085,-0.246256,-0.013203,0.027778


## 6) Calculate the Squared Error of Individual Samples

For real and shuffled data

In [12]:
y_df = pd.concat([y_train_df, y_test_df]).reindex(all_scores.index)

print(y_df.shape)
y_df.head(2)

(357, 70)


,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,cc_cc_edu_pos_mean,cc_cc_g1_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,0.040322,0.091643,0.043915,...,0.438339,0.059414,0.082424,0.000000,-0.065050,-0.020236,-0.007970,0.020263,0.408214,0.654575
profile_1,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,-0.182571,0.893453,-1.023968,...,0.067568,0.256141,0.132834,0.386327,0.575026,0.225091,0.220461,-0.224965,0.284962,0.567898


In [13]:
all_score_error = sample_squared_error(scores=all_scores, y=y_df)

all_score_error = (
    metadata_df.merge(all_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results", "all_model_sample_squared_error.tsv")
all_score_error.to_csv(file, sep='\t', index=False)

print(all_score_error.shape)
all_score_error.head(2)

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.044028,0.017303,0.003511,0.031350,0.022116,0.005359,0.011153,...,0.144362,0.007561,0.002166,0.012434,0.104383,0.014107,0.028027,0.014339,0.038384,0.058896
1,profile_1,A549,AKT1-2,0.000037,0.677876,0.013271,0.065826,0.037256,0.113557,0.003239,...,0.000858,0.010840,0.018835,0.020916,0.109876,0.002626,0.000297,0.002719,0.001419,0.001025


In [14]:
all_shuffle_score_error = sample_squared_error(scores=all_shuffle_scores, y=y_df)

all_shuffle_score_error = (
    metadata_df.merge(all_shuffle_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results", "all_model_sample_squared_error_shuffled.tsv")
all_shuffle_score_error.to_csv(file, sep='\t', index=False)

print(all_shuffle_score_error.shape)
all_shuffle_score_error.head()

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.087023,0.157982,0.061285,0.293954,0.018481,0.010850,0.097913,...,0.492389,0.027040,0.004678,0.056820,0.078095,0.070843,0.062338,0.071032,0.181052,0.392873
1,profile_1,A549,AKT1-2,0.028320,1.214538,0.000389,0.544112,0.001584,0.011198,0.183995,...,0.109516,0.001061,0.000748,0.022391,0.130047,0.000434,0.000938,0.000453,0.088902,0.291729
2,profile_2,A549,ARID1B-1,0.008878,1.122345,0.009291,0.240383,0.007829,0.046980,0.058892,...,0.132299,0.005704,0.009862,0.003708,0.082677,0.036256,0.055422,0.036409,0.178186,0.054623
3,profile_3,A549,ARID1B-2,0.125820,0.054051,0.123070,0.099555,0.564438,0.248669,0.000150,...,0.299392,0.006526,0.000110,0.013594,0.078108,0.109125,0.107504,0.109359,0.042878,0.036387
4,profile_4,A549,ATF4-1,14.244958,0.171875,9.137624,4.353669,3.710436,8.355813,9.445549,...,4.712250,0.008054,0.033003,0.001590,0.086914,0.021100,0.000027,0.021217,0.483410,0.014468
